In [1]:
import os
import cv2
import requests
import base64
import pickle
import numpy as np

In [2]:
dataset_path = 'G:\Study\\3\\term1\AI\Cars Dataset'
test_path = 'G:\Study\\3\\term1\AI\Cars Dataset\\test'
train_path = 'G:\Study\\3\\term1\AI\Cars Dataset\\train'
#อ่านไฟล์
dataset_file = os.listdir(dataset_path)
test_file = os.listdir(test_path)
train_file = os.listdir(train_path)
# print(dataset_file)

In [3]:
#เปลี่ยนยี่ห้อรถเป็นเลข
def setBrandNumber(brand):
    index = test_file.index(brand)
    return index
# print(setBrandNumber('Audi'))

In [4]:
#แปลงภาพเป็น base64
def setbase64(img):
    url = "http://localhost:5000/api/gethog"
    v, buffer = cv2.imencode(".jpg", img)
    img_str = base64.b64encode(buffer)
    data = "image data,"+str.split(str(img_str),"'")[1]
    response = requests.get(url, json={"img":data})
    return response.json()
# img = cv2.imread('G:\Study\\3\\term1\AI\\traincarmodel\\train\Audi\\1.jpg')
# print(setbase64(img))

In [5]:
def gethog(pathtrain):
    response = []
    amountData = 0
    for brandfile in os.listdir(pathtrain):
        amountcar = 0
        for imagefile in os.listdir(pathtrain+"\\"+brandfile):
            img_path = pathtrain+"\\"+brandfile+"\\"+imagefile
            img = cv2.imread(img_path)
            res = setbase64(img)
            hog = list(res["hog"])
            hog.append(setBrandNumber(brandfile))
            response.append(hog)
            amountcar=amountcar+1
            amountData=amountData+1
        print(str(brandfile)+" : "+str(amountcar))
    print("amountData : "+str(amountData))
    return response

In [6]:
cars_train = gethog(train_path)
cars_test = gethog(test_path)

Audi : 814
Hyundai Creta : 271
Mahindra Scorpio : 316
Rolls Royce : 311
Swift : 424
Tata Safari : 441
Toyota Innova : 775
amountData : 3352
Audi : 199
Hyundai Creta : 67
Mahindra Scorpio : 75
Rolls Royce : 74
Swift : 102
Tata Safari : 106
Toyota Innova : 190
amountData : 813


In [7]:
# ทำเป็นไฟล์ pkl
def savePkl(filename ,cars):        
    write_path = filename + ".pkl"
    pickle.dump(cars, open(write_path,"wb"))
    print("data preparation is done")

# โหลดข้อมูลของไฟล์ไป pkl   
def loadPkl(filename):
    dataset = pickle.load(open(filename + '.pkl','rb'))
    return dataset

In [8]:
savePkl('train_cars',cars_train)

data preparation is done


In [9]:
savePkl('test_cars',cars_test)

data preparation is done


In [10]:
dataset_train = loadPkl('train_cars')
print("Data train : ",len(dataset_train))
dataset_test = loadPkl('test_cars')
print("Data test : ",len(dataset_test))

Data train :  3352
Data test :  813


In [11]:
# แปลง list ของข้อมูล train เป็น array
train_arr = np.array(dataset_train)
x_train = train_arr[:,0:-1]
y_train = train_arr[:,-1]

In [12]:
# แปลง list ของข้อมูล test เป็น array
test_arr = np.array(dataset_test)
x_test = test_arr[:,0:-1]
y_test = test_arr[:,-1]

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [14]:
# เทรนข้อมูลโดยใช้ Decision Tree
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [15]:
# ทดสอบประสิทธิภาพ
y_pred = clf.predict(x_test)
print("Accuracy:",accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix : \n",confusion_matrix(y_test,y_pred))

Accuracy: 47.23247232472325
Confusion Matrix : 
 [[ 80  14  12  28  17  26  22]
 [  9  38   5   3   0   3   9]
 [ 13   4  25   8   1   9  15]
 [ 25   5   8   8   9   9  10]
 [  9   7   9   9  48   2  18]
 [ 15   2   3   5   7  66   8]
 [ 27  13   5   4  15   7 119]]


In [16]:
# บันทึก model file ด้วย pickle
path_model ='carbrandmodel.pkl'
pickle.dump(clf, open(path_model,'wb'))